In [13]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
from share import *
import config

import cv2
import einops
#import gradio as gr
import numpy as np
import torch
import random

from pytorch_lightning import seed_everything
from annotator.util import resize_image, HWC3
from annotator.hed import HEDdetector
from cldm.model import create_model, load_state_dict
from cldm.ddim_hacked import DDIMSampler

import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

/home/yuebing/.conda/envs/controlnet_hemy/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


logging improved.


In [2]:
from decimal import Decimal

In [2]:
# 打印可用 GPU 的数量
num_gpus = torch.cuda.device_count()
print(f"Number of available GPUs: {num_gpus}")

Number of available GPUs: 7


In [3]:
device = torch.device('cpu')
print(f'Using device: {device}')

Using device: cpu


In [3]:
device = torch.device('cuda:2')
print(device)  # 输出：device(type='cuda', index=1)
torch.cuda.set_device(device)
# 检查设备是否正确设置
if torch.cuda.is_available():
    print(f"Using device: {torch.cuda.get_device_name(device)}")
else:
    print("CUDA is not available.")

cuda:2
Using device: NVIDIA GeForce RTX 3090


In [4]:
if torch.cuda.is_available():
    print(f"Currently using GPU: {torch.cuda.current_device()}")
    print(f"GPU name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
    print(f"Total memory: {torch.cuda.get_device_properties(torch.cuda.current_device()).total_memory}")
else:
    print("No GPU is being used.")

Currently using GPU: 2
GPU name: NVIDIA GeForce RTX 3090
Total memory: 25430786048


In [6]:
version = '24511719'
epoch = '4'
step = '112594'
city_dic={'chicago': 'Chicago', 'dallas':'Dallas', 'la':'Los Angeles'}

In [7]:
version = '0'
epoch = '1'
step = '20397'
city_dic={'chicago': 'Chicago', 'dallas':'Dallas', 'la':'Los Angeles'}

In [6]:
version = '0'
epoch = '4'
step = '50994'
city_dic={'chicago': 'Chicago', 'dallas':'Dallas', 'la':'Los Angeles'}

In [6]:
version = '3'
epoch = '6'
step = '101877'
city_dic={'chicago': 'Chicago', 'dallas':'Dallas', 'la':'Los Angeles'}

In [30]:
version = '3'
epoch = '10'
step = '160093'
city_dic={'chicago': 'Chicago', 'dallas':'Dallas', 'la':'Los Angeles'}

In [41]:
version = '3'
epoch = '16'
step = '247417'
city_dic={'chicago': 'Chicago', 'dallas':'Dallas', 'la':'Los Angeles'}

In [32]:
version = '3'
epoch = '24'
step = '363849'
city_dic={'chicago': 'Chicago', 'dallas':'Dallas', 'la':'Los Angeles'}

In [5]:
version = '3'
epoch = '29'
step = '436619'
city_dic={'chicago': 'Chicago', 'dallas':'Dallas', 'la':'Los Angeles'}

In [6]:
model = create_model('./models/cldm_v15.yaml').cpu()

No module 'xformers'. Proceeding without it.
ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Loaded model config from [./models/cldm_v15.yaml]


In [8]:

# model.load_state_dict(load_state_dict('./models/control_sd15_ini.ckpt', location='cuda'))
# model.load_state_dict(load_state_dict('./lightning_logs/version_23691853/checkpoints/epoch=8-step=74483.ckpt', location='cuda'))
# model.load_state_dict(load_state_dict('./lightning_logs/version_23745120/checkpoints/epoch=2-step=24494.ckpt', location='cuda:1'))
# model.load_state_dict(load_state_dict('./lightning_logs/version_23786778/checkpoints/epoch=12-step=106144.ckpt', location='cuda'))
# model.load_state_dict(load_state_dict('./lightning_logs/version_23786984/checkpoints/epoch=17-step=98531.ckpt', location='cuda'))

model.load_state_dict(load_state_dict('./lightning_logs/version_'+version+'/checkpoints/epoch='+epoch+'-step='+step+'.ckpt', location='cpu'))
#model.load_state_dict(load_state_dict('./version_archive/version_'+version+'_e'+epoch+'/checkpoints/epoch='+epoch+'-step='+step+'.ckpt', location='cpu'))

model.to(device)

Loaded state_dict from [./lightning_logs/version_3/checkpoints/epoch=29-step=436619.ckpt]


ControlLDM(
  (model): DiffusionWrapper(
    (diffusion_model): ControlledUnetModel(
      (time_embed): Sequential(
        (0): Linear(in_features=320, out_features=1280, bias=True)
        (1): SiLU()
        (2): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (input_blocks): ModuleList(
        (0): TimestepEmbedSequential(
          (0): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (1): TimestepEmbedSequential(
          (0): ResBlock(
            (in_layers): Sequential(
              (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
              (1): SiLU()
              (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            )
            (h_upd): Identity()
            (x_upd): Identity()
            (emb_layers): Sequential(
              (0): SiLU()
              (1): Linear(in_features=1280, out_features=320, bias=True)
            )
            (out_layers): Sequential(
             

In [11]:
ddim_sampler = DDIMSampler(model)

In [12]:
# 检查 PyTorch 使用的设备
if torch.cuda.is_available():
    print(f"PyTorch is using GPU: {torch.cuda.current_device()}")
    print(f"GPU name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("PyTorch is not using any GPU.")

PyTorch is using GPU: 2
GPU name: NVIDIA GeForce RTX 3090


In [13]:
dir_hemy = '/home/yuebing/Mingyi/GenerativeUrbanDesign/Urban_Data/'
image_dir = '/home/yuebing/Mingyi/GenerativeUrbanDesign/Urban_Data/Vector_Image_Partition/'
city = 'NewYork'
data_dir = [dir_hemy + 'Descriptions/NewYork_grid_r0_d0.csv',
            dir_hemy + 'Descriptions/NewYork_grid_r1_d0.csv',
            dir_hemy + 'Descriptions/NewYork_grid_r2_d0.csv',
            dir_hemy + 'Descriptions/NewYork_grid_r0_d1.csv',
            dir_hemy + 'Descriptions/NewYork_grid_r1_d1.csv',
            dir_hemy + 'Descriptions/NewYork_grid_r2_d1.csv',
            dir_hemy + 'Descriptions/NewYork_grid_r0_d2.csv',
            dir_hemy + 'Descriptions/NewYork_grid_r1_d2.csv',
            dir_hemy + 'Descriptions/NewYork_grid_r2_d2.csv']

In [14]:
df = pd.DataFrame() 
for f in data_dir: 
    dfi = pd.read_csv(f)
    df = pd.concat([df, dfi[dfi['Train']==0]])

#desc = {}
#for c in ['NewYork']:
    #desc[c] = pd.read_csv("~/satellite_tiles_control/tile_descriptions/tile_descriptions_"+c+"_1114.csv").set_index(['xtile','ytile'])
    #desc[c] = pd.read_csv("/home/yuebing/Mingyi/GenerativeUrbanDesign/Urban_Data/Descriptions/"+

In [15]:
df.head()

,geometry,left_x,top_y,row,col,index_right,FID,r,d,fully_water_tag,...,lu_p_p,lu_mixed_p,lu_add,rd_pri_area,rd_pri_p,rd_res_p,rd_p,rd_category,descriptions,Train
9,POLYGON ((279415.7200310936 38577.641927330675...,278965.720031,39027.641927,9,2,0,0,r0,d0,0,...,0.0,0.0,0.806853,4418.146961,0.021818,0.046191,0.068009,low,Land use parcels include 37.5 percent of resid...,0
15,POLYGON ((279865.7200310936 37677.641927330675...,279415.720031,38127.641927,7,3,0,0,r0,d0,0,...,0.0,0.0,1.000000,3007.268568,0.014851,0.070089,0.084940,medium,Land use parcels include 100.0 percent of resi...,0
17,POLYGON ((279865.7200310936 38577.641927330675...,279415.720031,39027.641927,9,3,0,0,r0,d0,0,...,0.0,0.0,1.000000,3150.531579,0.015558,0.084412,0.099970,medium,Land use parcels include 79.1 percent of resid...,0
21,POLYGON ((279865.7200310936 40377.641927330675...,279415.720031,40827.641927,13,3,0,0,r0,d0,0,...,0.0,0.0,0.768221,4318.422732,0.021326,0.035199,0.056525,low,Land use parcels include 30.4 percent of resid...,0
25,POLYGON ((279865.7200310936 42177.641927330675...,279415.720031,42627.641927,17,3,0,0,r0,d0,0,...,0.0,0.0,0.358653,0.000000,0.000000,0.000000,0.000000,low,Land use parcels include 0.0 percent of reside...,0


In [15]:
import os
os.makedirs('./lightning_logs/version_'+version+'/image_log/', exist_ok=True)
os.makedirs('./lightning_logs/version_'+version+'/image_output/', exist_ok=True)

In [16]:
dir_hemy = '/home/yuebing/Mingyi/GenerativeUrbanDesign/Urban_Data/'
image_dir = dir_hemy + 'Vector_Image_Partition/'

In [16]:
def process(xtile, ytile, r, d, zoom, input_city, output_city, prompt, a_prompt, n_prompt, num_samples, image_resolution, detect_resolution, ddim_steps, guess_mode, strength, scale, seed, eta, v=None, RGB=True):
    
    with torch.no_grad():
        #input_image = cv2.imread("../satellite_tiles_control/satellite_tiles/"+str(zoom)+"/"+input_city+"/"+str(zoom)+"_"+str(x)+"_"+str(y)+".png")
        
        #if right == 0:
        #    if down == 0:
        #        hint_name = image_dir + city + '_stage_1/' + str(xtile) + '_' + str(ytile) + '_' + str(xcord) + '_' + str(ycord) + '.tif'
        #        img_name = image_dir + city + '_stage_2/' + str(xtile) + '_' + str(ytile) + '_' + str(xcord) + '_' + str(ycord) + '.tif'
        #    elif down == 1:
        #        hint_name = image_dir + city + '_stage_1_down/' + str(xtile) + '_' + str(ytile) + '_' + str(xcord) + '_' + str(ycord) + '.tif'
        #        img_name = image_dir + city + '_stage_2_down/' + str(xtile) + '_' + str(ytile) + '_' + str(xcord) + '_' + str(ycord) + '.tif'
        #elif right == 1:
        #    if down == 0:
        #        hint_name = image_dir + city + '_stage_1_right/' + str(xtile) + '_' + str(ytile) + '_' + str(xcord) + '_' + str(ycord) + '.tif'
        #        img_name = image_dir + city + '_stage_2_right/' + str(xtile) + '_' + str(ytile) + '_' + str(xcord) + '_' + str(ycord) + '.tif'
        #    elif down == 1:
        #        hint_name = image_dir + city + '_stage_1_right_down/' + str(xtile) + '_' + str(ytile) + '_' + str(xcord) + '_' + str(ycord) + '.tif'
        #        img_name = image_dir + city + '_stage_2_right_down/' + str(xtile) + '_' + str(ytile) + '_' + str(xcord) + '_' + str(ycord) + '.tif'  
        #
        
        hint_name = image_dir + city + '_Stage_1_' + r + '_' + d + '/' + str(xtile) + '_' + str(ytile) + '_' + r + '_' + d + '.tif'
        img_name =  image_dir + city + '_Stage_2_' + r + '_' + d + '/' + str(xtile) + '_' + str(ytile) + '_' + r + '_' + d + '.tif'     

        input_image = cv2.imread(img_name)
        input_image = np.array(input_image)
        print(img_name)
        H,W,C = input_image.shape

        #if v is None:
        #    detected_map = cv2.imread("../satellite_tiles_control/skeleton/"+str(zoom)+"/"+input_city+"/"+str(x)+"/"+str(y)+".png", cv2.IMREAD_UNCHANGED)
        #else:
        #    detected_map = cv2.imread("../satellite_tiles_control/skeleton/"+str(zoom)+"/test/"+str(x)+"_"+str(y)+"_"+v+".png", cv2.IMREAD_UNCHANGED)
        
        detected_map = cv2.imread(hint_name, cv2.IMREAD_UNCHANGED)
        detected_map = np.array(detected_map)    
        
        if detected_map.shape[2] == 4:
            # convert 4-channel source image to 3-channel
            #make mask of where the transparent bits are
            trans_mask = detected_map[:,:,3] == 0
    
            #replace areas of transparency with white and not transparent
            detected_map[trans_mask] = [255, 255, 255, 255]
    
            #new image without alpha channel...
            detected_map = cv2.cvtColor(detected_map, cv2.COLOR_BGRA2BGR)        
        
        #OpenCV read images in BGR order.
        control = cv2.cvtColor(detected_map, cv2.COLOR_BGR2RGB)
        control = torch.from_numpy(control.copy()).float().cuda() / 255.0
        control = torch.stack([control for _ in range(num_samples)], dim=0)
        control = einops.rearrange(control, 'b h w c -> b c h w').clone()

        if seed == -1:
            seed = random.randint(0, 65535)
        seed_everything(seed)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=False)
            
        cond = {"c_concat": [control], "c_crossattn": [model.get_learned_conditioning([prompt + ', ' + a_prompt] * num_samples)]}
        un_cond = {"c_concat": None if guess_mode else [control], "c_crossattn": [model.get_learned_conditioning([n_prompt] * num_samples)]}
        shape = (4, H // 8, W // 8)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=True)

        model.control_scales = [strength * (0.825 ** float(12 - i)) for i in range(13)] if guess_mode else ([strength] * 13)  # Magic number. IDK why. Perhaps because 0.825**12<0.01 but 0.826**12>0.01
        samples, intermediates = ddim_sampler.sample(ddim_steps, num_samples,
                                                     shape, cond, verbose=False, eta=eta,
                                                     unconditional_guidance_scale=scale,
                                                     unconditional_conditioning=un_cond)
            
        if config.save_memory:
            model.low_vram_shift(is_diffusing=False)

        x_samples = model.decode_first_stage(samples)
        x_samples = (einops.rearrange(x_samples, 'b c h w -> b h w c') * 127.5 + 127.5).cpu().numpy().clip(0, 255).astype(np.uint8)

        if RGB:
            results = [x_samples[i] for i in range(num_samples)]
            detected_map = cv2.cvtColor(detected_map, cv2.COLOR_BGR2RGB)
        else:
            results = [cv2.cvtColor(x_samples[i], cv2.COLOR_RGB2BGR) for i in range(num_samples)]

        return [detected_map] + [input_image] + results


In [17]:
zoom = "16"
num_samples = 1
image_resolution = 512
strength = 1.0
guess_mode = False
detect_resolution = 512
ddim_steps = 20
scale = 9.0
seed = 5354
eta = 0.0
a_prompt = 'best quality, extremely detailed'
n_prompt = 'longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'

In [ ]:
df.shape

In [ ]:
input_city = 'NewYork'
output_city = 'NewYork'

i = 0

xtile = df.iloc[i]['row']
ytile = df.iloc[i]['col']
right = df.iloc[i]['r']
down  = df.iloc[i]['d']

prompt = df.iloc[i]['descriptions']
#prompt = df.loc[(int(xtile),int(ytile))]['Land_use_description']

RGB=True
outputs0 = process(xtile, ytile, right, down, zoom, input_city, output_city, prompt, a_prompt, n_prompt, num_samples, image_resolution, detect_resolution, ddim_steps, guess_mode, strength, scale, seed, eta, v='0', RGB=RGB)
    

In [21]:
len(outputs0)

3

In [22]:
import random

data = list(range(7354))

# 随机选择30个不同的行号
random_indices = random.sample(data, 30)

selected_ranges = [range(index, index+1) for index in random_indices]

# 打印结果
for r in selected_ranges:
    print(r)

range(4873, 4874)
range(2043, 2044)
range(7119, 7120)
range(740, 741)
range(424, 425)
range(3627, 3628)
range(3270, 3271)
range(2934, 2935)
range(4604, 4605)
range(694, 695)
range(2859, 2860)
range(7202, 7203)
range(6899, 6900)
range(6184, 6185)
range(5304, 5305)
range(6843, 6844)
range(5619, 5620)
range(801, 802)
range(5881, 5882)
range(7080, 7081)
range(4391, 4392)
range(292, 293)
range(2845, 2846)
range(3910, 3911)
range(1979, 1980)
range(5539, 5540)
range(2195, 2196)
range(2670, 2671)
range(4959, 4960)
range(1519, 1520)


In [18]:
random_indices = [
3215,
4139,
9,
3631,
54,
4805,
3404,
6939,
7051,
857,
1070,
6765,
64,
2534,
7284,
4908,
6404,
6611,
5282,
5820,
5357,
294,
4840,
1630,
2616,
3351,
6560,
5906,
2720,
5567]

In [19]:
df.iloc[5567]

geometry           POLYGON ((316315.7200310936 53127.641927330675...
left_x                                                 315865.720031
top_y                                                   53577.641927
row                                                               42
col                                                               84
index_right                                                        0
FID                                                                0
r                                                                 r0
d                                                                 d2
fully_water_tag                                                    0
water_area                                                         0
rd_m_area                                                        0.0
rd_0_area                                                        0.0
rd_1_area                                                        0.0
rd_2_area                         

In [20]:
input_city = 'NewYork'
output_city = 'NewYork'
outputs = []

for i in random_indices:

    xtile = df.iloc[i]['row']
    ytile = df.iloc[i]['col']
    right = df.iloc[i]['r']
    down  = df.iloc[i]['d']
    
    prompt = df.iloc[i]['descriptions']
    #prompt = df.loc[(int(xtile),int(ytile))]['Land_use_description']
    
    RGB=True
    outputs_i = process(xtile, ytile, right, down, zoom, input_city, output_city, prompt, a_prompt, n_prompt, num_samples, image_resolution, detect_resolution, ddim_steps, guess_mode, strength, scale, seed, eta, v='0', RGB=RGB)
    outputs.append(outputs_i)

Global seed set to 5354


/home/yuebing/Mingyi/GenerativeUrbanDesign/Urban_Data/Vector_Image_Partition/NewYork_Stage_2_r0_d1/70_99_r0_d1.tif
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


DDIM Sampler: 100%|█████████████████████████████| 20/20 [00:07<00:00,  2.57it/s]
Global seed set to 5354


/home/yuebing/Mingyi/GenerativeUrbanDesign/Urban_Data/Vector_Image_Partition/NewYork_Stage_2_r2_d1/33_19_r2_d1.tif
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


DDIM Sampler: 100%|█████████████████████████████| 20/20 [00:05<00:00,  3.77it/s]
Global seed set to 5354


...

DDIM Sampler: 100%|█████████████████████████████| 20/20 [00:05<00:00,  3.86it/s]
Global seed set to 5354


/home/yuebing/Mingyi/GenerativeUrbanDesign/Urban_Data/Vector_Image_Partition/NewYork_Stage_2_r0_d1/56_59_r0_d1.tif
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


DDIM Sampler: 100%|█████████████████████████████| 20/20 [00:04<00:00,  4.10it/s]
Global seed set to 5354


/home/yuebing/Mingyi/GenerativeUrbanDesign/Urban_Data/Vector_Image_Partition/NewYork_Stage_2_r0_d2/42_84_r0_d2.tif
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


DDIM Sampler: 100%|█████████████████████████████| 20/20 [00:05<00:00,  3.87it/s]


In [21]:
def convert_bgr_to_rgb(image):
    """
    将 BGR 图像转换为 RGB 图像。
    """
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [22]:
import os
output_file = './output_image/epoch_' + epoch
os.makedirs(output_file, exist_ok=True)

In [23]:
for x in range(30):
    i = random_indices[x]
    xtile = df.iloc[i]['row']
    ytile = df.iloc[i]['col']
    right = df.iloc[i]['r']
    down = df.iloc[i]['d']
    
    for y in range(3):        
        if y ==2:
            image = outputs[x][y]
        
            # Save each image with the desired naming convention
            filename = f'{xtile}_{ytile}_{right}_{down}.png'
            plt.imsave(os.path.join(output_file,filename), image)
            

In [ ]:
fig, ax = plt.subplots(30,3, figsize=(6,72))
#fig, ax = plt.subplots(5,3, figsize=(6,12))
for x in range(30):
    for y in range(3):
        if y == 1:
            ax[x,y].imshow(convert_bgr_to_rgb(outputs[x][y]))
        else:
            ax[x,y].imshow(outputs[x][y])

for ax in ax.ravel():
    ax.set_axis_off()
#fig.savefig('output_image_e'+epoch+'.png', bbox_inches='tight')
plt.show()